_Do these steps on terminal_

In [ ]:
!pip install --upgrade pip && pip install hyperdash

In [ ]:
!hd signup

In [ ]:
!hyperdash login

# Monitor Your Machine Learning Experiment

In [ ]:
import numpy as np 
import pandas as pd 

disease = pd.read_csv('heart.csv')
disease.describe()

#Split the data into train and test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(disease, test_size = 0.2, random_state = 1)

X_train = train.drop(['target','fbs'],axis=1)
y_train = train['target']

X_test = train.drop(['target','fbs'],axis=1)
y_test = train['target']

In [ ]:
from hyperdash import Experiment
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

#Declare your Experiment object each run
exp = Experiment('SGDClassifier')

#Record the value of the hyperparameter alpha
alpha = exp.param('alpha', 0.01)

sgd_model = SGDClassifier(random_state=1, alpha = alpha)
sgd_model.fit(X_train, y_train)

#Evaluate with cross validation score
from sklearn.model_selection import cross_val_score

score = cross_val_score(sgd_model, X_train, y_train).mean()
exp.metric('Accuracy', score)

#Evaluate with confusion matrix

sgd_predictions = cross_val_predict(sgd_model, X_train, y_train)
tn, fp, fn, tp  = confusion_matrix(y_train, sgd_predictions).ravel()

exp.metric('True Negative', tn)
exp.metric('False Positive', fp)
exp.metric('False Negative', fn)
exp.metric('True Positive', tp) 

exp.end()


# Decorator experiment

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from hyperdash import monitor

@monitor('RandomForestClassifier')
def random_forest(exp):
    
    n_estimators = 100
    forest_clf = RandomForestClassifier(random_state=1, n_estimators= n_estimators)
    forest_predictions = cross_val_predict(forest_clf, X_train, y_train )
    f1 = metrics.f1_score
    exp.metric('f1', f1)

# Observer metrics

In [ ]:
from hyperdash import Experiment
exp = Experiment("Dogs vs. Cats")

# Parameters
estimators = exp.param("Estimators", 500)
epochs = exp.param("Epochs", 5)
batch = exp.param("Batch Size", 64)

for epoch in xrange(1, epochs + 1):
    accuracy = 1. - 1./epoch
    loss = float(epochs - epoch)/epochs
    print("Training model (epoch {})".format(epoch))
    time.sleep(1)

    # Metrics
    exp.metric("Accuracy", accuracy)
    exp.metric("Loss", loss)

exp.end()